$\textbf{ROI slection}$<br>
$\text{In this workflow we crop the data temporaly and specialy. We also calculate the raw mc, pulse per ions and ion per pulse. }$

In [42]:
# Activate intractive functionality of matplotlib
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import numpy as np
from ipywidgets import fixed
from ipywidgets import interact_manual
from ipywidgets import widgets

# Local module and scripts
from pyccapt.calibration.calibration_tools import tools, share_variables
from pyccapt.calibration.calibration_tools import widgets as wd
from pyccapt.calibration.data_tools import data_tools, data_loadcrop, dataset_path_qt
from pyccapt.calibration.mc import mc_tools, tof_tools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
# Disable showing logging in Jupyter notebook
# logging.disable(sys.maxsize)

In [44]:
button = widgets.Button(
    description='load dataset',
)
@button.on_click
def open_file_on_click(b):
    """
    Event handler for button click event.
    Prompts the user to select a dataset file and stores the selected file path in the global variable dataset_path.
    """
    global dataset_path
    dataset_path = dataset_path_qt.gui_fname().decode('ASCII')
button

Button(description='load dataset', style=ButtonStyle())

In [ ]:
# check pytables library and install it
# try:
#     import tables
# except:
#     !conda install --yes --prefix {sys.prefix} pytables

In [45]:
# create object for selection of instrument specifications of the dataset
tdc, pulse_mode, flightPathLength_d, t0_d, max_mc, det_diam = wd.dataset_instrument_specification_selection()

# Display lists and comboboxes to selected instrument specifications
display(tdc, pulse_mode, flightPathLength_d, t0_d, max_mc)

Dropdown(description='Data mode:', options=('PyCCAPT', 'leap_epos', 'leap_pos', 'ato_v6'), value='PyCCAPT')

Dropdown(description='Pulse mode:', options=('voltage', 'laser'), value='voltage')

FloatText(value=110.0, description='Flight path length:')

FloatText(value=54.0, description='t0:')

FloatText(value=400.0, description='Max mc:')

In [79]:
# Calculate the maximum possible time of flight (TOF)
max_tof = int(tof_tools.mc2tof(max_mc.value, 1000, 0, 0, flightPathLength_d.value))
print('The maximum possible TOF is:', max_tof, 'ns')
print('=============================')
# create an instance of the Variables opject
variables = share_variables.Variables()
variables.pulse_mode = pulse_mode.value
dataset_main_path = os.path.dirname(dataset_path)
dataset_name_with_extention = os.path.basename(dataset_path)
variables.dataset_name = os.path.splitext(dataset_name_with_extention)[0]
variables.result_data_path = dataset_main_path + '/' + variables.dataset_name 
variables.result_data_name = 'cropped_' +  variables.dataset_name 
variables.result_path = dataset_main_path + '/' + variables.dataset_name + '/load_crop/'

if not os.path.isdir(variables.result_path):
    os.makedirs(variables.result_path, mode=0o777, exist_ok=True)

print('The data will be saved on the path:', variables.result_data_path)
print('=============================')
print('The dataset name after saving is:', variables.result_data_name)
print('=============================')
print('The figures will be saved on the path:', variables.result_path)
print('=============================')

# Create data farame out of hdf5 file dataset
dld_group_storage = data_tools.load_data(dataset_path, tdc.value, mode='raw')

# Remove the data with tof greater thatn Max TOF or below 0 ns
data = data_tools.remove_invalid_data(dld_group_storage, max_tof)
print('Total number of Ions:', len(data))

The maximum possible TOF is: 5010 ns
The data will be saved on the path: D:/pyccapt/tests/data/data_1647_Sep-01-2023_16-58_Pd_35_unchargedV2
The dataset name after saving is: cropped_data_1647_Sep-01-2023_16-58_Pd_35_unchargedV2
The figures will be saved on the path: D:/pyccapt/tests/data/data_1647_Sep-01-2023_16-58_Pd_35_unchargedV2/load_crop/
{'apt': ['high_voltage', 'main_chamber_vacuum', 'num_events', 'pulse', 'temperature', 'time_counter'], 'dld': ['high_voltage', 'pulse', 'start_counter', 't', 'x', 'y'], 'tdc': ['channel', 'high_voltage', 'pulse', 'start_counter', 'time_data'], 'time': ['time_h', 'time_m', 'time_s']}
The number of data over max_tof: 22
Total number of Ions: 652847


In [80]:
data

,high_voltage (V),pulse,start_counter,t (ns),x_det (cm),y_det (cm)
0,571.250000,328.0,11183,4197.034180,0.800000,0.796735
1,579.755005,328.0,6580,2988.051270,1.557551,-2.191020
2,585.440002,328.0,46,2795.279541,-2.948571,0.355918
3,585.440002,328.0,3408,2468.352051,2.749388,-0.911020
4,593.952515,328.0,2453,3560.700928,0.277551,0.741224
...,...,...,...,...,...,...
652842,10000.000000,1500.0,15003,1361.244385,-1.900408,-0.884898
652843,10000.000000,1500.0,15984,3638.628418,1.488980,0.355918
652844,10000.000000,1500.0,17409,22.946869,-0.417959,-2.465306
652845,10000.000000,1500.0,17409,151.575516,-1.495510,-1.880816


$\textbf {Temporal crop}$
<br>
<br>
$\text {Select the data by drawing a rectangel over the experiment history}$

In [81]:
interact_manual(data_loadcrop.plot_crop_experiment_history, data=fixed(data), variables=fixed(variables), max_tof=widgets.FloatText(value=max_tof), frac=widgets.FloatText(value=1.0),
                bins=fixed((1200,800)), figure_size=fixed((7,3)),
               draw_rect=fixed(False), data_crop=fixed(True), pulse_plot=widgets.Dropdown(options=[('False', False), ('True', True)]), dc_plot=widgets.Dropdown(options=[('True', True), ('False', False)]),
                pulse_mode=widgets.Dropdown(options=[('voltage', 'voltage'), ('laser', 'laser')]), save=widgets.Dropdown(options=[('True', True), ('False', False)]),
               figname=widgets.Text(value='hist_ini'));


C:\Users\APTUser\AppData\Local\Temp\ipykernel_35000\4188335034.py:1: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(data_loadcrop.plot_crop_experiment_history, data=fixed(data), variables=fixed(variables), max_tof=widgets.FloatText(value=max_tof), frac=widgets.FloatText(value=1.0),


interactive(children=(FloatText(value=5010.0, description='max_tof'), FloatText(value=1.0, description='frac')…

$\text{Boundaries of the selected(cropped) part of the graph is printed below}$

In [84]:
# Plot and selected experiment history
interact_manual(data_loadcrop.plot_crop_experiment_history, data=fixed(data), variables=fixed(variables), max_tof=widgets.FloatText(value=max_tof), frac=widgets.FloatText(value=1.0),
                bins=fixed((1200,800)), figure_size=fixed((7,3)),
               draw_rect=fixed(True), data_crop=fixed(False), pulse_plot=widgets.Dropdown(options=[('False', False), ('True', True)]), dc_plot=widgets.Dropdown(options=[('True', True), ('False', False)]),
                pulse_mode=widgets.Dropdown(options=[('voltage', 'voltage'), ('laser', 'laser')]), save=widgets.Dropdown(options=[('True', True), ('False', False)]),
               figname=widgets.Text(value='hist_rect'));


C:\Users\APTUser\AppData\Local\Temp\ipykernel_35000\2323789099.py:2: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(data_loadcrop.plot_crop_experiment_history, data=fixed(data), variables=fixed(variables), max_tof=widgets.FloatText(value=max_tof), frac=widgets.FloatText(value=1.0),


interactive(children=(FloatText(value=5010.0, description='max_tof'), FloatText(value=1.0, description='frac')…

In [85]:
# Crop the dataset
print('Min Idx:', variables.selected_x1, 'Max Idx:', variables.selected_x2)
data_crop_t = data_loadcrop.crop_dataset(data, variables)

Min Idx: 8633.47761223017 Max Idx: 638839.1366620576


$\textbf {Spacial crop}$
<br>
<br>
$\text {Select the region of maximum concentration of Ions in the below plotted graph to utlize relevant data}$

In [86]:
# Plot and select the FDM
interact_manual(data_loadcrop.plot_crop_fdm, data=fixed(data_crop_t), variables=fixed(variables), frac=widgets.FloatText(value=1.0),
                bins=fixed((256,256)), figure_size=fixed((5,4)),
               draw_circle=fixed(False), data_crop=fixed(True), 
                save=widgets.Dropdown(options=[('True', True), ('False', False)]),
               figname=widgets.Text(value='fdm_ini'));

C:\Users\APTUser\AppData\Local\Temp\ipykernel_35000\1567814332.py:2: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(data_loadcrop.plot_crop_fdm, data=fixed(data_crop_t), variables=fixed(variables), frac=widgets.FloatText(value=1.0),


interactive(children=(FloatText(value=1.0, description='frac'), Dropdown(description='save', options=(('True',…

$\text {The region selected in the previous step is displayed below.}$

In [87]:
# plot selected area in FDM
interact_manual(data_loadcrop.plot_crop_fdm, data=fixed(data_crop_t), variables=fixed(variables), frac=widgets.FloatText(value=1.0),
                bins=fixed((256,256)), figure_size=fixed((5,4)),
                draw_circle=fixed(True), data_crop=fixed(False), 
                save=widgets.Dropdown(options=[('True', True), ('False', False)]),
               figname=widgets.Text(value='fdm_circle'));

C:\Users\APTUser\AppData\Local\Temp\ipykernel_35000\3314188428.py:2: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(data_loadcrop.plot_crop_fdm, data=fixed(data_crop_t), variables=fixed(variables), frac=widgets.FloatText(value=1.0),


interactive(children=(FloatText(value=1.0, description='frac'), Dropdown(description='save', options=(('True',…

In [88]:
# Crop the dataset
print('center x:', variables.selected_x_fdm, 'center y:', variables.selected_y_fdm)
print('Radios:', variables.roi_fdm)
if variables.roi_fdm > 0:
    data_crop_spatial = data_loadcrop.crop_data_after_selection(data_crop_t, variables)
else:
    print('select the data spacialy from cell below')

center x: 0.4798422093721526 center y: 0.25477478242933316
Radios: 2.966515095827778


d:\pyccapt\pyccapt\calibration\data_tools\data_loadcrop.py:312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Create a pandas dataframe from the cropped data.


$\text {The final selected data after processing is shown below.}$

In [89]:
# Crop and plot the dataset
interact_manual(data_loadcrop.plot_crop_fdm, data=fixed(data_crop_spatial), variables=fixed(variables), frac=widgets.FloatText(value=1.0),
                bins=fixed((256,256)), figure_size=fixed((5,4)),
               draw_circle=fixed(False), data_crop=fixed(False), 
                save=widgets.Dropdown(options=[('True', True), ('False', False)]),
               figname=widgets.Text(value='fdm'));

C:\Users\APTUser\AppData\Local\Temp\ipykernel_35000\2530233233.py:2: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(data_loadcrop.plot_crop_fdm, data=fixed(data_crop_spatial), variables=fixed(variables), frac=widgets.FloatText(value=1.0),


interactive(children=(FloatText(value=1.0, description='frac'), Dropdown(description='save', options=(('True',…

$\text {Calculate pulses since the last event pulse and ions per pulse.}$

In [90]:
pulse_pi, ion_pp = data_loadcrop.calculate_ppi_and_ipp(data_crop_spatial)

# add two calculated array to the croped dataset
data_crop_spatial['pulse_pi'] = pulse_pi.astype(np.uintc)
data_crop_spatial['ion_pp'] = ion_pp.astype(np.uintc)


d:\pyccapt\pyccapt\calibration\data_tools\data_loadcrop.py:359: RuntimeWarning: overflow encountered in ulong_scalars
  for j in range(multi_hit_count):
d:\pyccapt\pyccapt\calibration\data_tools\data_loadcrop.py:364: RuntimeWarning: overflow encountered in ulong_scalars
  previous_counter = current_counter
C:\Users\APTUser\AppData\Local\Temp\ipykernel_35000\949659944.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_crop_spatial['pulse_pi'] = pulse_pi.astype(np.uintc)
C:\Users\APTUser\AppData\Local\Temp\ipykernel_35000\949659944.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

$\text {The percentage of loss in ROI selection process.}$

In [91]:
# save the cropped data
print('tof Crop Loss {:.2f} %'.format((100 - (len(data_crop_spatial) / len(data)) * 100)))

tof Crop Loss 15.61 %


In [92]:
# exctract needed data from Pandas data frame as an numpy array
dld_highVoltage = data_crop_spatial['high_voltage (V)'].to_numpy()
dld_pulse = data_crop_spatial['pulse'].to_numpy()
dld_t = data_crop_spatial['t (ns)'].to_numpy()
dld_x = data_crop_spatial['x_det (cm)'].to_numpy()
dld_y = data_crop_spatial['y_det (cm)'].to_numpy()

In [93]:
def fine_tune_t_0(variables, t0_d, bin_size, mode, prominence, distance, percent, selector, plot, figname, lim, peaks_find_plot):
    variables.mc_calib = mc_tools.tof2mc(dld_t, t0_d, dld_highVoltage, dld_x, dld_y, flightPathLength_d.value, dld_pulse, mode=pulse_mode.value)
    tools.mc_hist_plot(variables, bin_size, mode, prominence, distance, percent, selector, plot, figname, lim, peaks_find_plot)
    
interact_manual(fine_tune_t_0, variables=fixed(variables), t0_d=widgets.FloatText(value=t0_d.value), bin_size=widgets.FloatText(value=0.1), 
                mode=widgets.Dropdown(options=[('mc', 'mc')]), prominence=widgets.IntText(value=50), distance=widgets.IntText(value=100), 
                lim=widgets.IntText(value=400), percent=widgets.IntText(value=50), selector=fixed('None'), plot=fixed(True), figname=widgets.Text(value='hist'),
                peaks_find_plot=fixed(True));

C:\Users\APTUser\AppData\Local\Temp\ipykernel_35000\820851973.py:5: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(fine_tune_t_0, variables=fixed(variables), t0_d=widgets.FloatText(value=t0_d.value), bin_size=widgets.FloatText(value=0.1),


interactive(children=(FloatText(value=27.0, description='t0_d'), FloatText(value=0.1, description='bin_size'),…

In [100]:
data_crop_spatial_back = data_crop_spatial.copy()

In [101]:
data_crop_spatial_back.insert(0, 'x (nm)', np.zeros(len(dld_t)))
data_crop_spatial_back.insert(1, 'y (nm)', np.zeros(len(dld_t)))
data_crop_spatial_back.insert(2,'z (nm)', np.zeros(len(dld_t)))
data_crop_spatial_back.insert(3,'mc_c (Da)', np.zeros(len(dld_t)))
data_crop_spatial_back.insert(4, 'mc (Da)', variables.mc_calib)
data_crop_spatial_back.insert(8,'t_c (ns)', np.zeros(len(dld_t)))

$\text {Remove the data with mc biger than max mc.}$

In [102]:
# Remove the data with mc biger than max mc
mask = (data_crop_spatial_back['mc (Da)'].to_numpy() > max_mc.value)
print('The number of data over max_mc:', len(mask[mask==True]))
data_crop_spatial_back.drop(np.where(mask)[0], inplace=True)
data_crop_spatial_back.reset_index(inplace=True, drop=True)

# Remove the data with x,y,t = 0
mask1 = (data_crop_spatial_back['x (nm)'].to_numpy() == 0)
mask2 = (data_crop_spatial_back['y (nm)'].to_numpy() == 0)
mask3 = (data_crop_spatial_back['t (ns)'].to_numpy() == 0)
mask = np.logical_and(mask1, mask2)
mask = np.logical_and(mask, mask3)
print('The number of data with having t, x, and y equal to zero is:', len(mask[mask==True]))
data_crop_spatial_back.drop(np.where(mask)[0], inplace=True)
data_crop_spatial_back.reset_index(inplace=True, drop=True)

The number of data over max_mc: 52871
The number of data with having t, x, and y equal to zero is: 0


In [103]:
data_crop_spatial_back

,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,0.0,0.0,0.0,0.0,58.906548,4751.734863,712.760254,15294,0.0,888.488220,0.156735,1.910204,0,0
1,0.0,0.0,0.0,0.0,59.479944,4751.734863,712.760254,16307,0.0,912.779236,2.827755,1.293061,1013,2
2,0.0,0.0,0.0,0.0,59.153511,4751.734863,712.760254,16327,0.0,881.479309,0.195918,-1.031837,20,1
3,0.0,0.0,0.0,0.0,61.227579,4751.734863,712.760254,17860,0.0,902.348206,1.270204,1.139592,1533,1
4,0.0,0.0,0.0,0.0,222.075121,4751.734863,712.760254,17861,0.0,1701.812744,-0.653061,-2.422857,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498032,0.0,0.0,0.0,0.0,58.856447,9779.985352,1466.997803,12296,0.0,628.494568,-0.088163,1.466123,2,1
498033,0.0,0.0,0.0,0.0,56.558688,9779.985352,1466.997803,12435,0.0,634.563904,2.654694,-1.668571,139,1
498034,0.0,0.0,0.0,0.0,58.635152,9779.985352,1466.997803,12504,0.0,626.183411,-0.186122,1.270204,69,1
498035,0.0,0.0,0.0,0.0,45.798670,9779.985352,1466.997803,13021,0.0,558.090332,-0.143673,-1.214694,517,1


In [104]:
data_crop_spatial_back.dtypes

x (nm)              float64
y (nm)              float64
z (nm)              float64
mc_c (Da)           float64
mc (Da)             float64
high_voltage (V)    float64
pulse               float64
start_counter        uint32
t_c (ns)            float64
t (ns)              float64
x_det (cm)          float64
y_det (cm)          float64
pulse_pi             uint32
ion_pp               uint32
dtype: object

In [105]:
interact_manual(data_tools.save_data, data=fixed(data_crop_spatial_back), variables=fixed(variables),
                hdf=widgets.Dropdown(options=[('True', True), ('False', False)]),
                epos=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                pos=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                ato_6v=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                csv=widgets.Dropdown(options=[('False', False), ('True', True)]));

interactive(children=(Dropdown(description='hdf', options=(('True', True), ('False', False)), value=True), Dro…